# Deep CORAL
**Main idea:**
Though CORAL is very simple and it works surprisingly well in some unsupervised domain adaptation cases, it has some drawbacks:
- CORAL lies on linear transformation.
- CORAL is not end-to-end training.  --> It needs to extract the feature from the source domain and target domain first to apply linear transformation. Then it trains a simple classifier (e.g. SVM,KNN) separately.

So we design Deep CORAL as below:
![image.png](images/2.png)

source数据的输出与source label进行监督训练，得到分类的loss。而target数据的输出由于没有标注数据进行监督训练，因此要和source进行适应，计算CORAL loss。最终的目的是要将分类loss和CORAL loss共同优化到最小，即source的分类更精确，target的输出与source的分布更相似。因此损失函数由两部分组成：
$$l = l_{CLASS} + \sum_{i=1}^t \lambda_i l_{CORAL}$$

Why:
- 单纯最小化分类loss会导致模型对源域过拟合，在目标域上性能很差
- 单纯对CORAL loss优化会恶化特征。    -->网络会将source和target数据映射，如果映射到了同一个点，CORAL loss会变为0，这样的特征是不能构建强大的分类器的。

**Cons of DeepCORAL:**
1. 学习非线性的变换，比CORAL更强大
2. 比DAN优化起来更容易
3. 可以无缝集成到CNN结构中

In [1]:
import os
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import time
from torchvision import models

In [ ]:
def load_data(root_path, domain, batch_size, phase):
    transform_dict = {
        'src': transforms.Compose(
        [transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ]),
        'tar': transforms.Compose(
        [transforms.Resize(224),
         transforms.ToTensor(),
         transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225]),
         ])}
    data = datasets.ImageFolder(root=os.path.join(root_path, domain), transform=transform_dict[phase])
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=phase=='src', drop_last=phase=='tar', num_workers=4)
    return data_loader